# Matrix Factorization(MF)

- 사용자와 아이템을 추상화된 동일한 특질(latent factor)을 이용하여 표현
- Rating matrics 구조를 표현하는데 SVD(Singular Value Decomposition)으로 user,item profile vector로 표현하여 rating을 예측하는 방법

<img width='1000' src='img/MF/MF기본구조.png'>


### Matrix Factorization: Rank - k 로 일반화

- 위 예시는 one rank 이지만 K개 차원으로 U,V를 각각 표현 가능

<img width='1000' src='img/MF/MFRank_K.png'>

# MF 다양한 학습 방법론

## SVD(Singular Value Decomposition)
<img width='800' src='img/MF/incompleteSVD.png'>

## Regularized SVD
<img width='800' src='img/MF/regularizedSVD.png'>

- 기존 SVD 함수에 규제를 적용한 방식으로 앞으로 기본적으로 적용되고 생략되어 설명된다

## Regularized SVD by Gradient Descent
<img width='800' src='img/MF/Regularized SVD by Gradient Descent.png'>

- U or V를 상수로 두고 Gradient Descent를 적용하여 업데이트
- 단점 속도가 느림

## Reqularized SVD by Alternating Least
<img width='800' src='img/MF/ALS_1.png'>
<img width='800' src='img/MF/ALS_2.png'>

- U or V 중 하나를 고정하고 linear regression방식으로 푸는 방식
- 모두 학습 대상 변수 (unknown variable): non-convex
- 둘 중 한 변수는 정해져 있다고 가정하면, quadratic problem (convex)
- 초기 임의의 값으로 U,V 중 하나를 설정하고 U,V가 수렴할 때까지 업데이트
- 초기 설정값에 따라 학습결과가 달라지지만 학습 속독 빠름

## Reqularized SVD: Bias Terms
<img width='800' src='img/MF/SVDBias.png'>

- 사용자와 아이템마다 평점 분포가 다르기 때문에 r를 단순히 U*V로 기술하는데 한계가 있음
- bias 항을 추가하여 표현하자

- b: 전체평점평균
- bi: 아이템i의평점편차
- bu: 사용자u의평점편차



## SVD++
<img width='800' src='img/MF/SVD++.png'>

# Matrix Factorization 장단점

장점
- 별점 예측이(메모리 기반 방식보다) 정확함
- 모델 기반 방식이라 일단 한번 학습만 끝나면 예측이 빠름

단점
- 학습시 많은 computing resource가 요구됨(1회성으로 큰 부담 안되는 경우가 많다)
- 추천하는 이유를 설명하기 어려움(가장 큰 단점)
